#### 🧍 Dim_Cliente

- PK: 
  - `cd_produto`

- Sequence By:
  - `ts_load`

- Descrição: 
  - Clientes únicos.

- Objetivo: 
  | Campo       | Tipo   |
  | ----------- | ------ |
  | cd_cliente  | INT    |
  | id_cliente  | STRING |

Autor: David Costa

## 01. Descrições/Parâmetros

In [0]:
# Definições de parâmetros e Descrição.
dicionario = {
    ## Destino.
    "catalog": "lakeflow",
    "schema": "d_gold",
    "prefix": "dim",
    "table": "cliente",
    ## Origem.
    "orCatalog": "lakeflow",
    "orSchema": "c_silver",
    "orTable": "slv_orders",
    # Informações.
    "PK": "id_cliente",
    "partition": "dt_partition",
    "sequence_by": "ts_load",
    "Descrição": "Clientes únicos."
}

# Cria o caminho de origem dos arquivos.
orPath = f'{dicionario["orCatalog"]}.{dicionario["orSchema"]}.{dicionario["orTable"]}'
print(f"Caminho de origem (Bronze): {orPath}")

# Cria o caminho de destino dos arquivos.
path = f'{dicionario["catalog"]}.{dicionario["schema"]}.{dicionario["prefix"]}_{dicionario["table"]}'
print(f"Caminho de destino (Silver): {path}")

## 01.2 Dicionário de Colunas

In [0]:
# Definições de parâmetros e Descrição.
orders_schema_dict = {
    "cd_cliente": {
        "datatype": "integer",
        "description": "Código único do cliente."
    },          
    "id_cliente": {
        "datatype": "string",
        "description": "Identificador do cliente no sistema."
    },
    "dt_partition": {
        "datatype": "Date",
        "description": "Data de partição."
    },
    "ts_load": {
        "datatype": "Timestamp",
        "description": "Data e hora de ingestão do registro no Data Lake."
    }
}

## 01.3 Importação de Bibliotecas

In [0]:
from pyspark.sql.functions import col, max, current_date, current_timestamp, monotonically_increasing_id
from delta.tables import DeltaTable

## 01.4 Configurações de Ambientes

In [0]:
# Verifica se a tabela de origem existe.
if not spark.catalog.tableExists(orPath):
    dbutils.notebook.exit("Execução abortada: Tabela origem não localizada.")

In [0]:
# Define se a carga será completa ou incremental.
boolean_carga_full = True

# Verifica se a tabela de destino já existe. Se existir, realiza carga incremental.
if spark.catalog.tableExists(path):
    boolean_carga_full = False

    # Retorna uma lista de objetos de partição
    partitions = spark.sql(f"SHOW PARTITIONS {path}")

    # Obtém a data da última partição criada.
    max_partition = partitions.select(max(dicionario["partition"])).first()[0]
    print(f"Carga incremental a partir da data: {max_partition}")

## 02. Leitura da Camada Silver

In [0]:
# Leitura das informações do arquivo CSV na camada Bronze.
if boolean_carga_full == False:
    # Carga incremental: lê apenas as partições mais recentes.
    df_silver = (
        spark.read \
        .table(orPath) \
        .filter(col("dt_partition") >= max_partition) \
        .select(
            col("customer_id").alias("id_cliente"),
        ).orderBy(col("product_id").asc())
    )
else:
    # Carga completa: lê todos os pedidos da tabela de origem.
    df_silver = (
        spark.read \
        .table(orPath) \
        .select(
            col("customer_id").alias("id_cliente"),
        ).orderBy(col("customer_id").asc())
    )

print(f"Quantidade de registros lidos: {df_silver.count()}")
# Exibe o esquema da tabela.
df_silver.printSchema()

## 03. Aplicação de filtros

In [0]:
# Remove duplicatas.
df_dropDuplicates = df_silver.dropDuplicates(['id_cliente'])
print(f"Quantidade de registros após remoção de duplicatas: {df_dropDuplicates.count()}")

## 04. Adicionando data e hora da carga

In [0]:
# Adiciona uma coluna com a data de ingestão.
df_ts = (
    df_dropDuplicates \
        .withColumn('dt_partition', current_date())
        .withColumn('ts_load', current_timestamp())
)

## 05. Cria/Atualiza Tabela

In [0]:
if boolean_carga_full:
    # Carga completa: sobrescreve a tabela Silver com todos os dados tratados.
    (
        df_ts \
            .withColumn("cd_cliente", monotonically_increasing_id()) \
            .write \
            .format("delta") \
            .mode("overwrite") \
            .partitionBy(dicionario['partition']) \
            .saveAsTable(path)
    )

else:
    # Carga incremental: realiza merge (upsert) dos dados tratados na tabela Silver existente.
    delta_table = DeltaTable.forName(spark, path)

    (
        delta_table.alias("t")
        .merge(
            source=df_ts.alias("s"),
            condition=f"t.{dicionario['PK']} = s.{dicionario['PK']}"
        )
        # Realiza atualização das linhas existentes e insere novas linhas.
        .whenMatchedUpdate(set={
            col: f"s.{col}" 
            for col in df_ts.columns if col not in [dicionario['PK'], 'cd_cliente']
        })
        .whenNotMatchedInsertAll()
        .execute()
    )

## 06. Adiciona Comentarios

## 06.1 Adiciona Comentarios: Tabela

In [0]:
# Adiciona as descrição na Tabela.    
spark.sql(f"COMMENT ON TABLE {path} IS '{dicionario['Descrição']}'")

## 06.2 Adiciona Comentarios: Colunas

In [0]:
# Adiciona as descrição na Tabela.
if len(orders_schema_dict):
    for field in orders_schema_dict:
        # Guarda a descrição da coluna.
        description = orders_schema_dict[field]['description']
        # Cria o comando SQL para adicionar a descrição.
        sql_cd = f"ALTER TABLE {path} ALTER COLUMN {field} COMMENT '{description}'"
        # Executa o comando SQL.
        spark.sql(sql_cd)